<a href="https://colab.research.google.com/github/Anacleto98/ANNDL_projects/blob/master/ANNDL_homework3/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np 
import os

SEED = 1234

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [26]:
working_directory = 'D:\\ANNDL_Challenge3_Dataset'
dataset_dir = os.path.join(working_directory,'VQA_Dataset')

print(os.listdir(dataset_dir))


['Images', 'test_questions.json', 'train_questions_annotations.json']


In [58]:
# importing json as python dictionaries 

import json 

with open(os.path.join(dataset_dir,'test_questions.json')) as json_file:
  test_question = json.load(json_file)

with open(os.path.join(dataset_dir,'train_questions_annotations.json')) as json_file:
  train_questions_annotations = json.load(json_file)

image_labels_ = os.listdir(os.path.join(dataset_dir,'Images'))
image_labels = []

for name in image_labels_:
  image_labels.append(name[:-4])


print(test_question[image_labels[2]])
print(image_labels[2])

questions_set = set()
answers_set = set()
answers_set_train = set()

for key in train_questions_annotations:
  questions_set.add(train_questions_annotations[key]['question'] + ' <eos>')
  answers_set.add(train_questions_annotations[key]['answer'] + ' <eos>')
  answers_set_train.add('<sos> ' + train_questions_annotations[key]['answer'])

{'question': 'Is the woman on the couch sporting white hair?', 'image_id': '1'}
10


In [59]:
# number of answers 

num_answers = len(answers_set)

# longest question is 100 chars
max_string = max(questions_set, key=len)
num_words = len(max_string.split(' '))

num_answers # number of classes 

words = set()
for question in questions_set:
  for w in question.split(' '):
    words.add(w)

print(len(words))

MAX_NUM_WORDS = 10000



6692


In [64]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# dictionary of training:  train_questions_annotations
# structure: 
questions_tokenizer = Tokenizer(num_words = MAX_NUM_WORDS,filters = '?')
answers_tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)

questions_tokenizer.fit_on_texts(questions_set)
answers_tokenizer.fit_on_texts(answers_set.union(answers_set_train))

questions_tokenized = questions_tokenizer.texts_to_sequences(questions_set)
answers_tokenized = answers_tokenizer.texts_to_sequences(answers_set)
answers_tokenized_train = answers_tokenizer.texts_to_sequences(answers_set_train)


# all sentences need a padding

max_length_question = max(len(sentence) for sentence in questions_tokenized)
max_length_answer_train = max(len(sentence) for sentence in answers_tokenized_train)
max_length_answer = max(len(sentence) for sentence in answers_tokenized)    

print(max_length_question)
print(max_length_answer)
print(max_length_answer_train)

# pad sequences
question_encoder_inputs = pad_sequences(questions_tokenized, maxlen=max_length_question)
answer_encoder_inputs = pad_sequences(answers_tokenized_train,maxlen=max_length_answer_train)
answer_outputs = pad_sequences(answers_tokenized,maxlen=max_length_answer,padding='post')

22


In [69]:
encoded_train_dictionary = {}

for key in train_questions_annotations:
  encoded_train_dictionary[key] = {}
  encoded_train_dictionary[key]['question'] = questions_tokenizer.texts_to_sequences(train_questions_annotations[key]['question']) 
  encoded_train_dictionary[key]['image_id'] = train_questions_annotations[key]['image_id']
  encoded_train_dictionary[key]['answer']   = answers_tokenizer.texts_to_sequences(train_questions_annotations[key]['answer'] + ' <eos>')
  encoded_train_dictionary[key]['answer_train'] = answers_tokenizer.texts_to_sequences('<sos> ' + train_questions_annotations[key]['answer'])



In [71]:
# not right

with open(os.path.join(working_directory,'encoded.json'), 'w') as fp:
    json.dump(encoded_train_dictionary, fp)